<a href="https://colab.research.google.com/github/2020Nina/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot
Un chatbot es un software inteligente que es capaz de comunicarse y realizar acciones similares a las de un humano. El objetivo de este proyecto es construir un modelo que prediga respuestas utilizando patrones y respuestas predefinidos. Se le proporciona un archivo llamado intents.json que contiene estos patrones. Los archivos de palabras y clases se proporcionan como ayuda adicional. Siéntase libre de crear un bot más complejo ampliando el archivo de intenciones.
#### Possible chat with your bot
<code>
You: Hello, how are you?

Bot: Hi there, how can I help?

You: what can you do?

Bot: I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies

You: thanks

Bot: My pleasure

You: see ya. got to go!

Bot: See you
</code>

In [ ]:
# Librerias que voy a utilizar
import pandas as pd
import json
import pickle
import urllib
import requests
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

En esta sección, se importaron las bibliotecas necesarias para realizar diversas tareas en el script. Algunas de estas bibliotecas incluyen urllib para manejar URL y cargar datos, pickle para serializar y deserializar objetos Python, random para la generación de números aleatorios, numpy para operaciones numéricas, y las bibliotecas relacionadas con scikit-learn (DecisionTreeClassifier, CountVectorizer, y make_pipeline) para construir y entrenar el modelo de árbol de decisiones.

In [ ]:
# Carga de datos
classes = pickle.load(urllib.request.urlopen("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectPCB/data/classes.pkl"))
intents = json.loads(requests.get("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectPCB/data/intents.json").text)

In [ ]:

intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [ ]:
# Crear un DataFrame de pandas para facilitar el procesamiento
df_intents = pd.DataFrame(intents['intents'])

# Separar los patrones y las respuestas en columnas separadas
df_patterns = df_intents['patterns'].apply(lambda x: pd.Series(x) if x else pd.Series(dtype=str)).stack().reset_index(level=1, drop=True).to_frame('pattern')
df_responses = df_intents['responses'].apply(lambda x: pd.Series(x) if x else pd.Series(dtype=str)).stack().reset_index(level=1, drop=True).to_frame('response')

# Unir las columnas de patrones y respuestas con el DataFrame original
df_intents = df_intents.join(df_patterns).join(df_responses)

# Eliminar filas con valores nulos en las columnas 'pattern' y 'response'
df_intents.dropna(subset=['pattern', 'response'], inplace=True)

# Resetear índice
df_intents.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame resultante
print(df_intents)

                tag                                           patterns  \
0          greeting  [Hi there, How are you, Is anyone there?, Hey,...   
1          greeting  [Hi there, How are you, Is anyone there?, Hey,...   
2          greeting  [Hi there, How are you, Is anyone there?, Hey,...   
3          greeting  [Hi there, How are you, Is anyone there?, Hey,...   
4          greeting  [Hi there, How are you, Is anyone there?, Hey,...   
..              ...                                                ...   
86  hospital_search  [Lookup for hospital, Searching for hospital t...   
87  hospital_search  [Lookup for hospital, Searching for hospital t...   
88  hospital_search  [Lookup for hospital, Searching for hospital t...   
89  hospital_search  [Lookup for hospital, Searching for hospital t...   
90  hospital_search  [Lookup for hospital, Searching for hospital t...   

                                            responses  \
0   [Hello, thanks for asking, Good to see you aga... 

In [ ]:
# Vectorizar el texto
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_intents['pattern'])

# Variables dependientes e independientes
Y = df_intents['tag']

# Dividir el conjunto de datos
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler(with_mean=False, with_std=True)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir la matriz dispersa a densa
X_train_scaled_dense = X_train_scaled.toarray()

In [ ]:
# Inicializar clasificadores
knn_classifier = KNeighborsClassifier()
svm_classifier = SVC()
dt_classifier = DecisionTreeClassifier()
nb_classifier = GaussianNB()
rf_classifier = RandomForestClassifier()

# Crear un clasificador de votación
voting_classifier = VotingClassifier(estimators=[
    ('knn', knn_classifier),
    ('svm', svm_classifier),
    ('dt', dt_classifier),
    ('nb', nb_classifier),
    ('rf', rf_classifier)
], voting='hard')

# Lista de clasificadores
clasificadores = [knn_classifier, svm_classifier, dt_classifier, nb_classifier, rf_classifier, voting_classifier]

# Realizar validación cruzada con KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for clf in clasificadores:
    # Obtener las predicciones de la validación cruzada
    Y_pred_cv = cross_val_predict(clf, X_train_scaled_dense, Y_train, cv=kf)

    # Calcular y mostrar la matriz de confusión
    cm = confusion_matrix(Y_train, Y_pred_cv)
    print(f"Clasificador: {clf.__class__.__name__}")
    print("Matriz de Confusión:")
    print(cm)

    # Calcular y mostrar la precisión, recall y F1-score
    precision = precision_score(Y_train, Y_pred_cv, average='macro', zero_division=1)
    recall = recall_score(Y_train, Y_pred_cv, average='macro')
    f1 = f1_score(Y_train, Y_pred_cv, average='macro')

    print(f"Precisión: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print("=" * 40)

# Entrenar el clasificador de votación con los datos de entrenamiento
voting_classifier.fit(X_train_scaled_dense, Y_train)

Clasificador: KNeighborsClassifier
Matriz de Confusión:
[[ 0  0  0  3  0  0  0  0  1]
 [ 0  0  1  0  0  0  0  0  2]
 [ 0  0  8  0  0  0  0  0  2]
 [ 0  0  0  4  0  0  0  0  7]
 [ 0  0  0  6  1  0  0  0  9]
 [ 0  0  1  1  0  0  0  0  2]
 [ 0  0  0  3  0  0  0  0  6]
 [ 0  0  0  2  0  0  0  0  2]
 [ 0  0  0  0  0  0  0  0 11]]
Precisión: 0.8080478975215817
Recall: 0.24734848484848485
F1-score: 0.17771200723475974
Clasificador: SVC
Matriz de Confusión:
[[ 0  0  0  0  0  0  4  0  0]
 [ 0  0  2  0  0  0  1  0  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  0  0  6  5  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0]
 [ 1  0  0  0  0  2  1  0  0]
 [ 0  0  0  0  0  0  9  0  0]
 [ 1  0  0  0  2  0  0  1  0]
 [ 0  0  0  0  1  0  0  0 10]]
Precisión: 0.7888888888888889
Recall: 0.5782828282828283
F1-score: 0.5760023201199672
Clasificador: DecisionTreeClassifier
Matriz de Confusión:
[[ 2  0  0  0  1  0  1  0  0]
 [ 0  0  3  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  0  0  7  4  0  0  0  0]
 [ 0  0  0 

VotingClassifier(estimators=[('knn', KNeighborsClassifier()), ('svm', SVC()),
                             ('dt', DecisionTreeClassifier()),
                             ('nb', GaussianNB()),
                             ('rf', RandomForestClassifier())])

In [ ]:
def train_and_evaluate(classifier, name, X_train, Y_train, X_test, Y_test):
    # Entrenar el clasificador
    classifier.fit(X_train, Y_train)

    # Obtener las predicciones en el conjunto de prueba
    Y_pred = classifier.predict(X_test)

    # Calcular y mostrar la matriz de confusión
    cm = confusion_matrix(Y_test, Y_pred)
    print(f"Modelo: {name}")
    print("Matriz de Confusión:")
    print(cm)

    # Calcular y mostrar la precisión, recall y F1-score
    precision = precision_score(Y_test, Y_pred, average='macro', zero_division=1)
    recall = recall_score(Y_test, Y_pred, average='macro')
    f1 = f1_score(Y_test, Y_pred, average='macro')

    print(f"Precisión: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print("=" * 40)

In [ ]:
# Inicializar y entrenar modelos
train_and_evaluate(knn_classifier, 'KNeighborsClassifier', X_train_scaled, Y_train, X_test_scaled, Y_test)
train_and_evaluate(svm_classifier, 'SVC', X_train_scaled, Y_train, X_test_scaled, Y_test)
train_and_evaluate(dt_classifier, 'DecisionTreeClassifier', X_train_scaled, Y_train, X_test_scaled, Y_test)
train_and_evaluate(rf_classifier, 'RandomForestClassifier', X_train_scaled, Y_train, X_test_scaled, Y_test)
train_and_evaluate(voting_classifier, 'VotingClassifier', X_train_scaled_dense, Y_train, X_test_scaled.toarray(), Y_test)

Modelo: KNeighborsClassifier
Matriz de Confusión:
[[0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4]
 [0 0 0 0 0 0 0 0 5]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 4]]
Precisión: 0.8024691358024691
Recall: 0.1111111111111111
F1-score: 0.0404040404040404
Modelo: SVC
Matriz de Confusión:
[[0 0 0 0 1 0 0 0 0]
 [0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 4]]
Precisión: 0.8148148148148149
Recall: 0.5555555555555556
F1-score: 0.5084175084175084
Modelo: DecisionTreeClassifier
Matriz de Confusión:
[[1 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 4]]
Precisión: 0.8888888888888888
Recall: 0.8333333333333334
F1-score: 0.8518518518518517


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo: RandomForestClassifier
Matriz de Confusión:
[[1 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 4]]
Precisión: 0.8888888888888888
Recall: 0.8333333333333334
F1-score: 0.8518518518518517


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo: VotingClassifier
Matriz de Confusión:
[[1 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 4]]
Precisión: 0.8888888888888888
Recall: 0.8333333333333334
F1-score: 0.8518518518518517


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
while True:
    # Obtener la entrada del usuario
    user_input_text = input("Usuario: ")

    # Salir del bucle si el usuario ingresa 'salir'
    if user_input_text.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Vectorizar la entrada del usuario
    user_input = vectorizer.transform([user_input_text])

    # Convertir la matriz dispersa a densa
    user_input_dense = user_input.toarray()

    # Escalar la entrada del usuario
    user_input_scaled = scaler.transform(user_input_dense)

    # Obtener la predicción del clasificador de votación
    user_prediction = voting_classifier.predict(user_input_scaled)

    # Obtener la respuesta asociada a la predicción
    response_index = classes.index(user_prediction[0])

    # Obtener la respuesta asociada a la predicción
    predicted_class = user_prediction[0]
    if predicted_class in classes:
        response_index = classes.index(predicted_class)
        if response_index in df_intents.index:
            bot_response = df_intents.loc[df_intents.index == response_index, 'response'].values[0]
            print("Chatbot:", bot_response)
        else:
            print("Chatbot: Lo siento, no tengo una respuesta para eso.")
    else:
        print("Chatbot: Clase predicha no encontrada en la lista de clases.")

Usuario: hello how are you?
Chatbot: Good to see you again
Usuario: what can you do?
Chatbot: Hello, thanks for asking
